In [1]:
import pickle
import os
import numpy as np

SAVED_MODEL_PATH = 'saved_model'
path_saved_model = os.path.join(os.getcwd(),SAVED_MODEL_PATH)
loaded_vectorizer = pickle.load(open(os.path.join(path_saved_model,'chatbot_tfidf_vectorizer.pkl'),'rb'))
loaded_model = pickle.load(open(os.path.join(path_saved_model,'chatbot_model.pkl'),'rb'))

In [2]:
new_complaint = 'help help help help cannot access my invoice'
x = loaded_vectorizer.transform([new_complaint])
res = loaded_model.predict_proba(x)[0]
print(res)
# print(loaded_model.classes_)

[0.01758826 0.0105855  0.9465413  0.00891716 0.01636778]


In [4]:
from typing import List, Dict

EXIT_CHAT = 'exit'
model = ''

MAP_INDEX_TO_LABEL = {
    0 : 'ACCOUNT',
    1 : 'CONTACT',
    2 : 'INVOICES',
    3 : 'ORDER',
    4 : 'PAYMENT'
}
# ['ACCOUNT' 'CONTACT' 'INVOICES' 'ORDER' 'PAYMENT']
THRESHOLD = {
    'ACCOUNT' : 0.8,
    'CONTACT' : 0.8,
    'INVOICES' : 0.8,
    'ORDER' : 0.8,
    'PAYMENT' : 0.8
}

def preprocess_text(text):
    return text

def vectorizer(vectorizer):
    return vectorizer

def inference(model, vectorizer, text) -> dict:
    if not model:
        # dummy value if model not exist
        inference_result = [0,0,0,0.2,0.8]
    else:
        x = vectorizer.transform([text])
        inference_result = model.predict_proba(x)[0]

    max_proba = inference_result.max()
    index_result = inference_result.argmax(axis=0)
    
    max_label = MAP_INDEX_TO_LABEL[index_result]
    

    return {'max_proba' : max_proba, 'max_label' : max_label} # ex {'max_proba' : 0.8, 'max_label' : 'ACCOUNT' }


def is_above_threshold(inference_result : dict) -> bool:
    if inference_result['max_proba'] > THRESHOLD[inference_result['max_label']]:
        return True
    else:
        return False
    
def exit_chat(text : str) -> bool:
    if text == EXIT_CHAT:
        return True
    else:
        return False

def short_chat_rule(text : str) -> bool:
    if len(text.split()) < 4:
        return True
    else:
        return False 

def main(model, vectorizer):
    flag = True
    while(flag):
        user_input = input()
        if user_input == 'exit':
            flag = False
        if flag:
            print(f'user : {user_input}')
            if not short_chat_rule(user_input):
                inference_result = inference(model,vectorizer,user_input)
                if is_above_threshold(inference_result):
                    response = inference_result['max_label']
                else:
                    response = 'below_proba'
            else:
                response = 'short_chat'
            print(f'bot : {response}')


if __name__ == "__main__":
    
    # load your model
    
    main(loaded_model, loaded_vectorizer)

user : need to cancel my order help
bot : ORDER
user : i cant access my account
bot : ACCOUNT
user : help
bot : short_chat
user : just an unnecessay comment
bot : below_proba


In [ ]:
# exit to exit chat